In [ ]:
import pandas as pd
import numpy as np

#Data Upload and Viewing

In [ ]:
# load airbnb data
gz = pd.read_csv('http://data.insideairbnb.com/united-states/ca/los-angeles/2021-04-07/data/listings.csv.gz')

In [ ]:
print(gz.shape)
gz.head()

In [ ]:
# remove unnecessary columns
cols = gz.columns
gz.columns

In [ ]:
# create a list of the columns to save
save = [5,6,27,29,30,35,36, 40,41]
cols_to_save = []

for i in save:
  cols_to_save.append(cols[i])

print(type(cols_to_save[0]))

cols_to_save

In [ ]:
# create a new dataframe with only the needed columns
df = gz[cols_to_save].copy()
df.head()

In [ ]:
# check data types for all columns
for i in df.columns:
  des = type(df[i][1])
  print(i)
  print(des)

# Data Pre-Processing

In [ ]:
df.isnull().sum()

In [ ]:
# fill null values
df['description'] = df['description'].fillna("None")
df['neighborhood_overview'] = df['neighborhood_overview'].fillna("None")
df['bathrooms_text'] = df['bathrooms_text'].fillna("1")
df['bedrooms'] = df['bedrooms'].fillna(1)

In [ ]:
# see if I missed any
df.isnull().sum()

In [ ]:
import re
def fix_bathroom(text):
  """
  Removes anything that is not a number
  Converts and returns the remaining number from a str to float
  """
  text = text.lower()

  if text[-9:] == 'half-bath':
    baths = float('0.5')
  else:
    baths = float(re.sub(r'[^0-9\.]', '', text))

  return baths

def to_float(num):
  return(float(num))


def fix_price(price_str):
  """
  convert price column from string to int
  """

  price = re.sub(r'[^0-9]', '', price_str)
  price = price[:-2]

  return int(price)

In [ ]:
# adjust columns to appropriate data format
df['bathrooms'] = df['bathrooms_text'].apply(fix_bathroom)
df['minimum_nights'] = df['minimum_nights'].apply(to_float)
df['target'] = gz['price'].apply(fix_price)

In [ ]:
# remove outliers
df = df[(df['target'] >= 25) & (df['target'] <= 2000) & 
        (df['bathrooms'] >=1) & (df['bathrooms'] < 4) &
        (df['bedrooms'] >= 1) & (df['bedrooms'] <= 4) &
        (df['minimum_nights']>=1) & (df['minimum_nights']<=30)]
df.shape

#Model

In [ ]:
df.columns

In [ ]:
df.drop(columns = ['description', 'neighborhood_overview', 'bathrooms_text'], inplace=True)

In [ ]:
df['minimum_nights'].describe()

In [ ]:
df.columns

In [ ]:
X = df.drop(columns=['target','latitude','longitude', 'neighbourhood_cleansed'])
y = df['target']

In [ ]:
!pip install category_encoders==2.*

In [ ]:
# one hot encode
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names = True)

X = encoder.fit_transform(X)

In [ ]:
X.columns

In [ ]:


# # # define base model
# def baseline_model():

#   # create model
#   model = Sequential()
#   model.add(Dense(5, input_dim=3, kernel_initializer='normal', activation='relu'))
#   model.add(Dense(6, kernel_initializer='normal', activation='relu'))
#   model.add(Dense(1, kernel_initializer='normal'))

#   # Compile model
#   model.compile(loss='mean_squared_error', optimizer='adam')

#   return model

# # evaluate model with standardized dataset
# estimators = []
# estimators.append(('standardize', StandardScaler()))
# estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)))
# pipeline = Pipeline(estimators)
# kfold = KFold(n_splits=5)
# results = cross_val_score(pipeline, X, y, cv=kfold)
# print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
def create_model():

  # create model
  model = Sequential()
  model.add(Dense(5, input_dim=3, kernel_initializer='normal', activation='relu'))
  model.add(Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')

  return model

# # evaluate model with standardized dataset
# estimators = []
# estimators.append(('standardize', StandardScaler()))
# estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
# pipeline = Pipeline(estimators)
# kfold = KFold(n_splits=5)
# results = cross_val_score(pipeline, X, y, cv=kfold)
# print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
model = create_model()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

model = Sequential()
model.add(Dense(5, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(6, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.25, random_state=42)

In [ ]:
type(y)

In [ ]:
model.fit(X, y,
          epochs = 122,
          batch_size=16,
          validation_data=(X_test, y_test)
          # metric='accuracy'
          )

In [ ]:
y[0:5]

In [ ]:
X.head(5)

In [ ]:
model.predict([[2,30,730,2]])

In [ ]:
model.predict([[4, 30, 1, 2]])

In [ ]:
# zip the model
!zip -r ./nn.zip ./nn_model/

In [ ]:
# download to local machine
from google.colab import files
files.download("./nn.zip")

In [ ]:
# test uploading saved model
from tensorflow import keras
model = keras.models.load_model('./nn_model')

prediction = model.predict([[2,30,730,2]])
prediction[0][0].round()

#Natural Language Processing

In [ ]:
# Start NLP 
from collections import Counter

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# NLP Libraries
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
# Initialize spacy model & tokenizer

nlp = spacy.load('en_core_web_lg')
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
# Create tokenize function

def tokenize(text):
    
    tokens = re.sub(r'[^a-zA-Z ^0-9]', ',', text)
    tokens = tokens.lower().replace(',', ' ')
    tokens = tokens.split()
    
    return tokens

def fix_bathroom(text):
  """
  Removes anything that is not a number
  Converts and returns the remaining number from a str to float
  """
  text = text.lower()

  if text[-9:] == 'half-bath':
    baths = float('0.5')
  else:
    baths = float(re.sub(r'[^0-9\.]', '', text))

  return baths


def fix_price(price_str):
  """
  convert price column from string to int
  """

  price = re.sub(r'[^0-9]', '', price_str)
  price = price[:-2]

  return int(price)

In [ ]:
df.columns

In [ ]:
# convert str prices to int prices
gz['target'] = gz['price'].apply(fix_price)

In [ ]:
token_cols

In [ ]:
type(df['bathrooms_text'][0])

In [ ]:
df['bathrooms']

In [ ]:
# tokenize dataframe
token_cols = df.columns

# remove numerical columns
token_cols = token_cols.drop(['latitude','bedrooms','longitude','bathrooms', 'minimum_nights'])

token_cols
for i in token_cols:
  if i == 'bathrooms_text':
    # df['bathrooms'] = df[i].apply(fix_bathroom)
    g=1+1
  else:
    df[i+"_token"] = df[i].apply(tokenize)

In [ ]:
df[['bedrooms', 'bathrooms', 'latitude', 'longitude']].describe()

In [ ]:
# remove outliers
df['target'] = gz['target']
# df = df[(df['target'] >= 25) & (df['target'] <= 2000) 
#       & (df['bathrooms'] <= 4) & (df['bathrooms'] >= 1)]
#       # (df['bedrooms'] <= 4) & (df['bedrooms'] >= 1)]

df = df[(df['target'] >= 25) & (df['target'] <= 2000) & 
        (df['bathrooms'] >=1) & (df['bathrooms'] < 4) &
        (df['bedrooms'] >= 1) & (df['bedrooms'] <= 4)]

In [ ]:
df.head()

In [ ]:
# Counter Function - takes a corpus of document and returns dataframe of word counts

from collections import Counter
 
word_counts = Counter()

def count(docs):

        word_counts = Counter()
        appears_in = Counter()
        
        total_docs = len(docs)

        for doc in docs:
            word_counts.update(doc)
            appears_in.update(set(doc))

        temp = zip(word_counts.keys(), word_counts.values())
        
        wc = pd.DataFrame(list(temp), columns = ['word', 'count'])

        wc['rank'] = wc['count'].rank(method='first', ascending=False)
        total = wc['count'].sum()

        wc['pct_total'] = wc['count'].apply(lambda x: x / total)
        
        wc = wc.sort_values(by='rank')
        wc['cul_pct_total'] = wc['pct_total'].cumsum()

        t2 = zip(appears_in.keys(), appears_in.values())
        ac = pd.DataFrame(t2, columns=['word', 'appears_in'])
        wc = ac.merge(wc, on='word')

        wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)
        
        return wc

In [ ]:
wc_neighborhood = count(df['neighbourhood_cleansed_token'])

wc_neighborhood.sort_values(by='rank')

In [ ]:
wc_neighborhood.shape

In [ ]:
#Cumulative Distribution Plot

sns.lineplot(x='rank', y='cul_pct_total', data=wc_neighborhood);

In [ ]:
# inspect some descriptions

df['description'].iloc[0]

In [ ]:
df['description'].iloc[1]

In [ ]:
df['description'][0]

In [ ]:
def clean(text):
  text = (text
    .str.replace('<br /><br />',' ')
    .str.replace('<b>',' ')
    .str.replace('</b><br />',' ')
    # .str.replace('</b><br />',' ')
    .str.replace('*','')
  )

  return text

def clean_description(text):

  cleaned = re.sub(r'[^a-zA-Z]', ',', text)
  cleaned = cleaned.lower().replace(',', ' ')

  return cleaned

In [ ]:
df['cleaned_desc'] = df['description'].apply(clean_description)

In [ ]:
# inspect cleaned descriptions and look for stop words
df['cleaned_desc'].iloc[0]

In [ ]:
# Vectorization

from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate Vectorizer
tfidf = TfidfVectorizer(stop_words='english', 
                        max_features=5000)

# Create a vocabulary and tf-idf score per description
dtm = tfidf.fit_transform(df['cleaned_desc'])

# Get feature names to use as dataframe column headers
general_dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

general_dtm.head()

In [ ]:
general_dtm.shape

In [ ]:
df.shape

#Seasons

In [ ]:
cal = pd.read_csv('http://data.insideairbnb.com/united-states/ca/los-angeles/2021-04-07/data/calendar.csv.gz')

In [ ]:
print(cal.shape)
cal.head()

In [ ]:
cal[cal['listing_id']==35922]

In [ ]:
def date_to_season(date):
  """
  Takes the string of date and returns what season it is in.
  """

  season = ['Winter', 'Spring', 'Summer', 'Fall']
  date_num = int(date[5:7])
  # print(date_num)


  if date_num <=2 or date_num ==12:
    return season[0]

  elif date_num<=5:
    return season[1]

  elif date_num<=8:
    return season[2]

  else:
    return season[3]

  return season[i]

In [ ]:
date_to_season(cal['date'][0])

In [ ]:
cal.drop(labels = 'Season', axis = 1)

In [ ]:
cal['Season'] = cal['date'].apply(date_to_season)
cal.head()

In [ ]:
listed = cal.listing_id.unique()
len(listed)

In [ ]:
temp = cal[cal['listing_id']==35922]
temp

In [ ]:
seasons = temp.Season.unique()
season_str = np.array2string(seasons)
type(season_str)

In [ ]:
new_season = list()

new_season.append(season_str)

new_season

In [ ]:
def define_seasons(data):
  """
  takes the dataframe and changes the 
  """